In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
%reload_ext autoreload

{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-10-11T10:54:09.989071Z", "msg": "Event choose-service-name: calling handler <function handle_service_name_alias at 0x7ff4fb059320>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-10-11T10:54:09.989782Z", "msg": "Event choose-service-name: calling handler <function handle_service_name_alias at 0x7ff4fb059320>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-10-11T10:54:09.990377Z", "msg": "Event choose-service-name: calling handler <function handle_service_name_alias at 0x7ff4fb059320>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-10-11T10:54:09.991000Z", "msg": "Event choose-service-name: calling handler <function handle_service_name_alias at 0x7ff4fb059320>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-10-11T10:54:09.992202Z"

In [3]:
import json

with open('topic_testing/podcast_08.txt', 'rb') as f:
    request = json.load(f)

In [4]:
from main import handler

res = handler(request, None)
group = json.loads(res['body'])

[nltk_data] Downloading package wordnet to /tmp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /tmp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-10-08T13:10:07.608229Z", "msg": "getting feature vector from mind service"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-10-08T13:10:07.609413Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7ff4faffac20>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-10-08T13:10:07.610751Z", "msg": "Event before-call.lambda.Invoke: calling handler <function inject_api_version_header_if_needed at 0x7ff4faffd680>"}
{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-10-08T13:10:07.611919Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestRespon

{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-10-08T13:10:07.613723Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7ff4fad9ee10>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-10-08T13:10:07.615239Z", "msg": "Event choose-signer.lambda.Invoke: calling handler <function set_operation_specific_signer at 0x7ff4faffa170>"}
{"level": "debug", "filename": "auth.py", "lineno": 364, "module": "auth", "ts": "2019-10-08T13:10:07.668891Z", "msg": "Calculating signature using v4 auth."}
{"level": "debug", "filename": "auth.py", "lineno": 365, "module": "auth", "ts": "2019-10-08T13:10:07.669715Z", "msg": "CanonicalRequest:\nPOST\n/2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations\n\nhost:lambda.us-east-1.amazonaws.com\nx-amz-date:20191008T131007Z\nx-amz-invocation-type:RequestResponse\n\nhost;x-amz-

KeyboardInterrupt: 

In [ ]:
## visualization
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
#m_time = formatTime("2019-09-16T09:53:21Z", True)
m_time = formatTime("2019-09-30T10:08:00Z", True)
for i in group['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")


In [ ]:
import boto3
from boto3 import client
from botocore.client import Config
import numpy as np
import json
aws_config = Config(
        connect_timeout=60,
        read_timeout=300,
        retries={"max_attempts": 0},
        region_name="us-east-1",
    )
lambda_client = client("lambda", config=aws_config)

def get_pims_score(req):

    #if req_data is None:
    #    lambda_payload = {"body": input_list}
    #    print (json.dumps(lambda_payload))
    #else:
    #    lambda_payload = {"body": {"request": req_data, "text_input": input_list}}
        
    try:
        #logger.info("Invoking lambda function")
        invoke_response = lambda_client.invoke(
            FunctionName="pim",
            InvocationType="RequestResponse",
            Payload=json.dumps(req),
        )
        lambda_output = (
            invoke_response["Payload"].read().decode("utf8")
        )
        response = json.loads(lambda_output)
        status_code = response["statusCode"]
        response_body = response["body"]

        if status_code == 200:
            result = json.loads(response_body)['d2vResult'][0]['distance']
        return result
    except Exception as e:
        print (e)
    return False

In [ ]:
pim_result = {}
pim_request = {"contextId": request["body"]["contextId"], "mindId": "01daaqyn9gbebc92aywnxedp0c"}
for seg in request['body']['segments']:
    pim_request["segments"] = [seg]
    # get_pims_score({"body":pim_request})
    pim_result[seg["recordingId"]] =  get_pims_score({"body":pim_request})

In [ ]:
group

In [ ]:
topic_pim = {
}
group_result = {}
for keys in group['group'].keys():
    for seg in group['group'][keys]:
        group_result[seg['recordingId']] = keys

In [ ]:
ranked_pims = sorted([(k,v) for (k,v) in pim_result.items()], key= lambda kv: kv[1])

In [ ]:
used_topics = []
group_no = None
index = 0
for (rec_id, distance) in ranked_pims:
    if rec_id in group_result.keys():
        group_no = group_result[rec_id]
        
        if group_no not in used_topics:
            topic_pim[index] = group_no
            used_topics.append(group_no)
            index += 1

In [120]:
topic_pim

{0: '24', 1: '40', 2: '32', 3: '15'}

In [125]:
final_output = []
final_output = list(map(lambda x: group['group'][x] , topic_pim.values()))

In [126]:
final_output[0][0]

{'id': '8ad2cdf53c0c47ee97ecc9eb649af405',
 'originalText': "Ux Designers have a a lot on their plate. They research that wire user tests the list goes on and off if you're new to the game this can all feel very overwhelming well throw that here out the window because we're about to embark on a journey through the sixteenth end of the UX design process that before starting any successful uX project you'll need to be in tune with two things your user in your brain. I start with the user big question around this is what problems are they making once you can identify the problem, then you can begin to start working to towards the solution second is understanding the brain and how does the project you're are working on aligned with the branch mission and goal to summarize in this stage you wanna identify the problems facing the users as well as the core goals of the brand and see how they along as a designer you probably have a gun of a*sumptions about what the user wants or doesn't want b

In [129]:
users = []
for result in final_output:
    temp_users = []
    for seg in result:
        if seg['spokenBy'] not in temp_users:
            temp_users.append(seg['spokenBy'])
    users.append(temp_users)

In [130]:
users

[['31a3ba4761854ad9a041ddf1c4c6a1dc'],
 ['31a3ba4761854ad9a041ddf1c4c6a1dc'],
 ['31a3ba4761854ad9a041ddf1c4c6a1dc'],
 ['31a3ba4761854ad9a041ddf1c4c6a1dc']]

# Testing topic level pims

## Read json Request

In [37]:
import json

with open('topic_testing/sync_eng_10-08.txt', 'rb') as f:
    request = json.load(f)

## Get Groups for the request

In [38]:
from main import handler

res = handler(request, None)
group = json.loads(res['body'])

{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-10-11T12:36:24.502146Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-10-11T12:36:24.502866Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-10-11T12:36:24.503506Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-10-11T12:36:24.504073Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-10-11T12:36:24.504826Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-10-11T12:36:24.505353Z", "msg": "getting feature vector from mind service"}
{"level": 

{"level": "info", "filename": "grouper_segments.py", "lineno": 128, "module": "grouper_segments", "outlier threshold is : ": 0.8769879772352027, "ts": "2019-10-11T12:38:08.706736Z", "msg": "Outlier Score"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 128, "module": "grouper_segments", "outlier threshold is : ": 0.8769879772352027, "ts": "2019-10-11T12:38:08.707492Z", "msg": "Outlier Score"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 128, "module": "grouper_segments", "outlier threshold is : ": 0.8769879772352027, "ts": "2019-10-11T12:38:08.708092Z", "msg": "Outlier Score"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 128, "module": "grouper_segments", "outlier threshold is : ": 0.8769879772352027, "ts": "2019-10-11T12:38:08.708615Z", "msg": "Outlier Score"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 128, "module": "grouper_segments", "outlier threshold is : ": 0.8769879772352027, "ts": "2019-10-11T12:38:08.7096

{"level": "info", "filename": "grouper_segments.py", "lineno": 469, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["That's right? I this this meeting is possibly like a first make sync call after after that vocabulary. So we'll see how it is working and then we keep improving that."], "2019-10-08T06:22:32Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "d52251dd988a46f79c3fb1daaeb1ae23"], "segment1": [["Correct, although I mean I get the decisions that we made with a deep initial. Yeah, I but he leads to whatever is we had getting the attempts will wait from a this. So yeah, that's yeah, should load platform addition."], "2019-10-08T06:22:43Z", "62b6ae1d7f834b0bb2055f7c72bc3368", "df69a8fc234b44d1a1bf23f87a8eeaff"], "segment2": [["Traffic so have remote topic extraction. Yes. So I deploy the staging to and tested got a got it. So and last week I had been I some of function changes like shown for CPU. Yeah, so test that itself today got it and let pick something else some back to."],

{"level": "info", "filename": "grouper_segments.py", "lineno": 469, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["That's right? I this this meeting is possibly like a first make sync call after after that vocabulary. So we'll see how it is working and then we keep improving that."], "2019-10-08T06:22:32Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "d52251dd988a46f79c3fb1daaeb1ae23"], "segment1": [["Correct, although I mean I get the decisions that we made with a deep initial. Yeah, I but he leads to whatever is we had getting the attempts will wait from a this. So yeah, that's yeah, should load platform addition."], "2019-10-08T06:22:43Z", "62b6ae1d7f834b0bb2055f7c72bc3368", "df69a8fc234b44d1a1bf23f87a8eeaff"], "segment2": [["Traffic so have remote topic extraction. Yes. So I deploy the staging to and tested got a got it. So and last week I had been I some of function changes like shown for CPU. Yeah, so test that itself today got it and let pick something else some back to."],

{"level": "info", "filename": "grouper_segments.py", "lineno": 469, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["That's right? I this this meeting is possibly like a first make sync call after after that vocabulary. So we'll see how it is working and then we keep improving that."], "2019-10-08T06:22:32Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "d52251dd988a46f79c3fb1daaeb1ae23"], "segment1": [["Correct, although I mean I get the decisions that we made with a deep initial. Yeah, I but he leads to whatever is we had getting the attempts will wait from a this. So yeah, that's yeah, should load platform addition."], "2019-10-08T06:22:43Z", "62b6ae1d7f834b0bb2055f7c72bc3368", "df69a8fc234b44d1a1bf23f87a8eeaff"], "segment2": [["Traffic so have remote topic extraction. Yes. So I deploy the staging to and tested got a got it. So and last week I had been I some of function changes like shown for CPU. Yeah, so test that itself today got it and let pick something else some back to."],

{"level": "info", "filename": "grouper_segments.py", "lineno": 469, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["That's right? I this this meeting is possibly like a first make sync call after after that vocabulary. So we'll see how it is working and then we keep improving that."], "2019-10-08T06:22:32Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "d52251dd988a46f79c3fb1daaeb1ae23"], "segment1": [["Correct, although I mean I get the decisions that we made with a deep initial. Yeah, I but he leads to whatever is we had getting the attempts will wait from a this. So yeah, that's yeah, should load platform addition."], "2019-10-08T06:22:43Z", "62b6ae1d7f834b0bb2055f7c72bc3368", "df69a8fc234b44d1a1bf23f87a8eeaff"], "segment2": [["Traffic so have remote topic extraction. Yes. So I deploy the staging to and tested got a got it. So and last week I had been I some of function changes like shown for CPU. Yeah, so test that itself today got it and let pick something else some back to."],

{"level": "info", "filename": "grouper_segments.py", "lineno": 469, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["That's right? I this this meeting is possibly like a first make sync call after after that vocabulary. So we'll see how it is working and then we keep improving that."], "2019-10-08T06:22:32Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "d52251dd988a46f79c3fb1daaeb1ae23"], "segment1": [["Correct, although I mean I get the decisions that we made with a deep initial. Yeah, I but he leads to whatever is we had getting the attempts will wait from a this. So yeah, that's yeah, should load platform addition."], "2019-10-08T06:22:43Z", "62b6ae1d7f834b0bb2055f7c72bc3368", "df69a8fc234b44d1a1bf23f87a8eeaff"], "segment2": [["Traffic so have remote topic extraction. Yes. So I deploy the staging to and tested got a got it. So and last week I had been I some of function changes like shown for CPU. Yeah, so test that itself today got it and let pick something else some back to."],

{"level": "info", "filename": "grouper_segments.py", "lineno": 469, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["That's right? I this this meeting is possibly like a first make sync call after after that vocabulary. So we'll see how it is working and then we keep improving that."], "2019-10-08T06:22:32Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "d52251dd988a46f79c3fb1daaeb1ae23"], "segment1": [["Correct, although I mean I get the decisions that we made with a deep initial. Yeah, I but he leads to whatever is we had getting the attempts will wait from a this. So yeah, that's yeah, should load platform addition."], "2019-10-08T06:22:43Z", "62b6ae1d7f834b0bb2055f7c72bc3368", "df69a8fc234b44d1a1bf23f87a8eeaff"], "segment2": [["Traffic so have remote topic extraction. Yes. So I deploy the staging to and tested got a got it. So and last week I had been I some of function changes like shown for CPU. Yeah, so test that itself today got it and let pick something else some back to."],

In [ ]:
request

## Get PIMs for the request

In [39]:
import boto3
from boto3 import client
from botocore.client import Config
import numpy as np
import json
aws_config = Config(
        connect_timeout=60,
        read_timeout=300,
        retries={"max_attempts": 0},
        region_name="us-east-1",
    )
lambda_client = client("lambda", config=aws_config)

def get_pims_score(req):

    #if req_data is None:
    #    lambda_payload = {"body": input_list}
    #    print (json.dumps(lambda_payload))
    #else:
    #    lambda_payload = {"body": {"request": req_data, "text_input": input_list}}
        
    try:
        #logger.info("Invoking lambda function")
        invoke_response = lambda_client.invoke(
            FunctionName="pim",
            InvocationType="RequestResponse",
            Payload=json.dumps(req),
        )
        lambda_output = (
            invoke_response["Payload"].read().decode("utf8")
        )
        response = json.loads(lambda_output)
        status_code = response["statusCode"]
        response_body = response["body"]

        if status_code == 200:
            result = json.loads(response_body)['d2vResult'][0]['distance']
        return result
    except Exception as e:
        print (e)
    return False

In [40]:
pim_result = {}
pim_response = {}
pim_request = {"contextId": request["body"]["contextId"], "mindId": "01DAAYHEKY5F4E02QVRJPTFTXV"}
pim_response = {"contextId": request["body"]["contextId"], "mindId": "01DAAYHEKY5F4E02QVRJPTFTXV", "segments": []}
temp = {}
for seg in request['body']['segments']:
    pim_request["segments"] = [seg]
    # get_pims_score({"body":pim_request})
    pim_result[seg["recordingId"]] =  get_pims_score({"body":pim_request})
    temp = seg
    temp["distance"] = pim_result[seg["recordingId"]]
    pim_response["segments"].append(temp)

## Extract topic level pims

In [41]:
import sys
sys.path.append("../")

import extract_topic_pims

In [42]:
from extract_topic_pims.main import handler

res = handler({"body":{"groups": group["group"], "pims": pim_response}}, None)
final_pims = json.loads(res)["body"]

{'1c31bac470354ff8bcbe8f22891875bf': '0', 'e3c9db4916004d7fbc85bbdca80a0b35': '42', '944dbbd78ce249549e31d9e8a6acf09d': '0', '09b303e0fa5443fe839c686d902405d1': '0', '1326531d0a98475daacc91081267e7cb': '0', '551978ff5da34f9b9523fc59baf74664': '0', 'a66604d2540943c0b54a399db1d91e7e': '42', 'e506483898cd4e89a25e7e7a8f9a8db5': '0', 'ee24b02ff2d349a38ae49945a6a25956': '1', '4da3395a9c354a339231ace8d2e8e20b': '1', '9cc53e7c4d6b4b2aad69fcf2f7d7b30c': '1', 'c6c8da4c507040f5be952195c8fdde42': '1', 'a968c453acad4eb1ac18c82a3dea31d4': '1', '126a1dddda7448d6b9954cd9e57d91fd': '1', 'faa66d4335a04d2cbe769ec62fe5e08d': '1', 'a01d9a11f3254eb9a87526219fe6888e': '1', '5f70631886374e9a9037d90c26893373': '1', 'd7bcdca4c01645aca5259766af7d0c3f': '31', 'a1f8ab7a8e414389b7934cb5d78196c6': '2', '444cc29f6ef74c9a95c7a3fca6de93d5': '38', 'fe52383583a44d7b97637503f810524d': '14', '80eec740eea74e60b3504c4f534b4408': '3', '42f2a690fba74ea99946c758afe4eec6': '3', '640efc6a4f554efd9ce32eb3014b94c3': '3', 'bb6c04c60

In [43]:
len(group['group']['76'])

KeyError: '76'

In [50]:
user_id_map = {}
user_id_map = {"b4a57b25de68446cac990f856d3fe4d5":"Deep Moradia","84fbaa66a2474ea29ae053f3a2e519d6":"Mithun","75bdf310110b4b8fab88b16fafce920e":"Trishanth Diwate","b1e8787a9a1f4859ac11cbb6a8124fd9": "Venkata Dikshit", "fb52cb663aec4795aee38ccfd904d315":"Reagan Rewop", "81a3e15469374fceba1cf972faa209b2":"Arjun Kini", "ecfeeb757f0a4d47af1ebd513929264a":"Shubham", "62b6ae1d7f834b0bb2055f7c72bc3368":"Karthik Muralidharan", "1a21542584494fcaba957d768b595b80":"Vamshi Krishna", "7e7ccbba232d411aa95ad3f244a35f40":"Shashank", "65bb83952fb54409a4bb59bb707f1375":"Vani", "0bbbfe84c66145af8d0ffcd5258bba38":"Parshwa Nemi Jain"}

import sys
sys.path.append("../../../ai-engine/pkg/")

from graphrank.core import GraphRank
from graphrank.utils import GraphUtils, TextPreprocess

gr = GraphRank()
tp = TextPreprocess()
gu = GraphUtils()

def get_desc(sentence):
    original_tokens, pos_tuple, filtered_pos_tuple = tp.preprocess_text(sentence, filter_by_pos=True, stop_words=False)
    word_graph = gr.build_word_graph(graph_obj=None, input_pos_text=pos_tuple, window=4, preserve_common_words=False)
    normal_keyphrase = gr.get_keyphrases(word_graph, pos_tuple, post_process=True)
    desc_keyphrase = gr.get_keyphrases(word_graph, pos_tuple, descriptive=True, post_process_descriptive=True)
    desc_keyphrase = sorted(desc_keyphrase, key=lambda kv:kv[1], reverse=True)
    normal_kp = [phrase for phrase, score in normal_keyphrase]
    desc_kp = [phrase for phrase, score in desc_keyphrase]
    
    return normal_kp, desc_kp

In [51]:
for groupid in final_pims:
    user_list =[]
    seg_list = []
    keyphrase = []
    for segi in groupid:
        if segi['spokenBy'] not in user_list:
            user_list.append(segi['spokenBy'])
        seg_list.append(segi['originalText'])
        #keyphrase.append(get_desc(segi['originalText']))
    #print ("User", end=" ")
    print (*[user_id_map[user] for user in user_list], sep=", ", end=" ")
    print ("Discussed \n\n Text: ", *seg_list, "\n\n")
    print (*get_desc(" ".join(sent for sent in seg_list))[1][:5], sep=", ")
    print ( "\n\n\n\n")

Shashank, Karthik Muralidharan Discussed 

 Text:  So updates so bit from my side or so I have uploaded like I have populated graph with at least ten thousand notes or something like fight or the notice or something so staging to I think around three or four context. Now I'm playing on with the so I'm just checking this on updated data if a think about it it or not done if it were populated correctly or not and and then checking the queries for that to get like all this basic insights information. So once I like finish experimenting all this queries I'm planning to write like small API end points on top of this. So that we we just call them I get says get users with gives spoken words or something and we we just get the results kind of. This this one is just for local like basic testing kind of thing here so just to get the high level queries done. So we can get like user information user who's spoke about specific set of words kind of information. And we can get all words that user as

join my table time, lot one the time, second time, multiple time, work to suppose







In [46]:
user_id_map = {}
user_id_map = {"84fbaa66a2474ea29ae053f3a2e519d6":"Mithun","75bdf310110b4b8fab88b16fafce920e":"Trishanth Diwate","b1e8787a9a1f4859ac11cbb6a8124fd9": "Venkata Dikshit", "fb52cb663aec4795aee38ccfd904d315":"Reagan Rewop", "81a3e15469374fceba1cf972faa209b2":"Arjun Kini", "ecfeeb757f0a4d47af1ebd513929264a":"Shubham", "62b6ae1d7f834b0bb2055f7c72bc3368":"Karthik Muralidharan", "1a21542584494fcaba957d768b595b80":"Vamshi Krishna", "7e7ccbba232d411aa95ad3f244a35f40":"Shashank", "65bb83952fb54409a4bb59bb707f1375":"Vani", "0bbbfe84c66145af8d0ffcd5258bba38":"Parshwa Nemi Jain"}


# testing hierarchy community

In [101]:
import json

with open('topic_testing/podcast_28.txt', 'rb') as f:
    request = json.load(f)

In [102]:
from main import handler

res = handler(request, None)
group = json.loads(res['body'])

{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:02:28.314253Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:02:28.315016Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:02:28.315649Z", "msg": "getting feature vector from mind service"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.316552Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.317144Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename"

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T15:02:28.323420Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": [\\\\\"Indeed everyone they share back again with another video.\\\\\", \\\\\"Now in this video we will talk about what is a pi or also known as application programming interface.\\\\\", \\\\\"Now, this is a very common term very useful term.\\\\\", \\\\\"But there is a problem with our society or our computer Society be give XnumberXfXnumberX, whether people understand that subject or not.\\\\\", \\\\\"We just want it to be technically correct.\\\\\", \\\\\"So if you look up for the API definition o

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T15:02:28.325038Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": [\\\\\"Indeed everyone they share back again with another video.\\\\\", \\\\\"Now in this video we will talk about what is a pi or also known as application programming interface.\\\\\", \\\\\"Now, this is a very common term very useful term.\\\\\", \\\\\"But there is a problem with our society or our computer Society be give XnumberXfXnumberX, whether people understand that subject or not.\\\\\", \\\\\"We just want it to be technically correct.\\\\\", \\\\\"So if you look up for the API definition o

{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.326537Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.327136Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.327728Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.328450Z", "msg": "Event choose-signer.lambda.Invoke: calling handler <f

{"level": "debug", "filename": "connectionpool.py", "lineno": 396, "module": "connectionpool", "ts": "2019-09-30T15:04:06.271674Z", "msg": "https://lambda.us-east-1.amazonaws.com:443 \"POST /2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations HTTP/1.1\" 200 4595271"}
{"level": "debug", "filename": "connectionpool.py", "lineno": 396, "module": "connectionpool", "ts": "2019-09-30T15:04:06.272984Z", "msg": "https://lambda.us-east-1.amazonaws.com:443 \"POST /2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations HTTP/1.1\" 200 4595271"}
{"level": "debug", "filename": "connectionpool.py", "lineno": 396, "module": "connectionpool", "ts": "2019-09-30T15:04:06.273614Z", "msg": "https://lambda.us-east-1.amazonaws.com:443 \"POST /2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations HTTP/1.1\" 200 4595271"}
{"level": "debug", "filename": "parsers.py", "lineno": 234, "module": "parsers", "ts": "2019-09-30T15:04:06.274688Z", "msg": "Response headers: {'Date': 'Mon

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic."], "2019-09-28T05:34:40Z", "31a3ba4761854ad9a041ddf1c4c6a1dc", "4e865bf0f59b440db31c432cf51347b7"], "segment1": [["Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and a

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic."], "2019-09-28T05:34:40Z", "31a3ba4761854ad9a041ddf1c4c6a1dc", "4e865bf0f59b440db31c432cf51347b7"], "segment1": [["Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and a

In [105]:
## visualization
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
m_time = formatTime("2019-09-28T05:34:21Z", True)
#m_time = formatTime("2019-09-30T10:28:00Z", True)
for i in group['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")





PIMs  0


Discussion:

 
Minutes from the start of the meeting:  0:00:19 4e865bf0f59b440db31c432cf51347b7 

Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic. 

Minutes from the start of the meeting:  0:00:58 a0349078dbc746199b273b5a3b3e8f51 

Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and after watching this video you never ever watch a

In [106]:
groups = []
for i in group['group'].keys():
    if len(group['group'][i])==1:
        continue
    else:
        temp = []
        for seg in group['group'][i]:
            temp.append(seg['originalText'])
        groups.append(' '.join(temp))

In [107]:
group_result = {}
#group_response = {}
group_request = {"contextId": request["body"]["contextId"], "mindId": "01daaqyn9gbebc92aywnxedp0c", "instanceId": "xyz"}
#group_response = {"contextId": request["body"]["contextId"], "mindId": "01DADP74WFV607KNPCB6VVXGTG", "segments": []}
temp = {}
group_itr = None
for segments_id in group['group'].keys():
    if len(group['group'][segments_id]) > 2:
        group_request['segments'] = group['group'][segments_id]
        res = handler({"body":group_request}, None)
        group_itr = json.loads(res['body'])
        break

{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:16:21.833136Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:16:21.833949Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:16:21.834604Z", "msg": "getting feature vector from mind service"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.835641Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.836270Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename"

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T15:16:21.841634Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": [\\\\\"Everyone they share back again with another video.\\\\\", \\\\\"We will talk about what is a pi or also known as Catholic.\\\\\", \\\\\"Indeed everyone they share back again with another video.\\\\\", \\\\\"Now in this video we will talk about what is a pi or also known as application programming interface.\\\\\", \\\\\"Now, this is a very common term very useful term.\\\\\", \\\\\"But there is a problem with our society or our computer Society be give XnumberXfXnumberX, whether people underst

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T15:16:21.842960Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": [\\\\\"Everyone they share back again with another video.\\\\\", \\\\\"We will talk about what is a pi or also known as Catholic.\\\\\", \\\\\"Indeed everyone they share back again with another video.\\\\\", \\\\\"Now in this video we will talk about what is a pi or also known as application programming interface.\\\\\", \\\\\"Now, this is a very common term very useful term.\\\\\", \\\\\"But there is a problem with our society or our computer Society be give XnumberXfXnumberX, whether people underst

{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.844338Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.845103Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.845809Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.846798Z", "msg": "Event choose-signer.lambda.Invoke: calling handler <f

{"level": "debug", "filename": "connectionpool.py", "lineno": 396, "module": "connectionpool", "ts": "2019-09-30T15:17:31.608069Z", "msg": "https://lambda.us-east-1.amazonaws.com:443 \"POST /2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations HTTP/1.1\" 200 2153774"}
{"level": "debug", "filename": "parsers.py", "lineno": 234, "module": "parsers", "ts": "2019-09-30T15:17:31.609441Z", "msg": "Response headers: {'Date': 'Mon, 30 Sep 2019 15:18:13 GMT', 'Content-Type': 'application/json', 'Content-Length': '2153774', 'Connection': 'keep-alive', 'x-amzn-RequestId': '326d4a56-be3d-4ab5-8724-72d3bb2f89af', 'x-amzn-Remapped-Content-Length': '0', 'X-Amz-Executed-Version': '$LATEST', 'X-Amzn-Trace-Id': 'root=1-5d921c6f-7cdeec60f9fc47486cd2f448;sampled=0'}"}
{"level": "debug", "filename": "parsers.py", "lineno": 234, "module": "parsers", "ts": "2019-09-30T15:17:31.610493Z", "msg": "Response headers: {'Date': 'Mon, 30 Sep 2019 15:18:13 GMT', 'Content-Type': 'application/json', 'Conten

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic."], "2019-09-28T05:34:40Z", "31a3ba4761854ad9a041ddf1c4c6a1dc", "4e865bf0f59b440db31c432cf51347b7"], "segment1": [["Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and a

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic."], "2019-09-28T05:34:40Z", "31a3ba4761854ad9a041ddf1c4c6a1dc", "4e865bf0f59b440db31c432cf51347b7"], "segment1": [["Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and a

In [108]:
## visualization
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
#m_time = formatTime("2019-09-16T09:53:21Z", True)
m_time = formatTime("2019-09-30T10:28:00Z", True)
m_time = formatTime("2019-09-28T05:34:21Z", True)
for i in group_itr['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group_itr['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")





PIMs  0


Discussion:

 
Minutes from the start of the meeting:  0:00:19 4e865bf0f59b440db31c432cf51347b7 

Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic. 

Minutes from the start of the meeting:  0:00:58 a0349078dbc746199b273b5a3b3e8f51 

Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and after watching this video you never ever watch a

In [89]:
group_result = {}
#group_response = {}
group_request = {"contextId": request["body"]["contextId"], "mindId": "01DADP74WFV607KNPCB6VVXGTG", "instanceId": "xyz"}
#group_response = {"contextId": request["body"]["contextId"], "mindId": "01DADP74WFV607KNPCB6VVXGTG", "segments": []}
temp = {}
group_itr_2 = None
for segments_id in group_itr['group'].keys():
    if len(group_itr['group'][segments_id]) > 2:
        group_request['segments'] = group_itr['group'][segments_id]
        res = handler({"body":group_request}, None)
        group_itr_2 = json.loads(res['body'])
        break

{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T14:29:07.140392Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T14:29:07.141156Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T14:29:07.142037Z", "msg": "getting feature vector from mind service"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.142868Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.143833Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename"

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T14:29:07.149214Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01dadp74wfv607knpcb6vvxgtg/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": \\\\\" Okay I think my join let us just started with that community on the language models right yep so. Dont how any with having but engine General communities. Yeah, it is was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally does not behavior.

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T14:29:07.150331Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01dadp74wfv607knpcb6vvxgtg/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": \\\\\" Okay I think my join let us just started with that community on the language models right yep so. Dont how any with having but engine General communities. Yeah, it is was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally does not behavior.

{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.151367Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.152272Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.152928Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.153688Z", "msg": "Event choose-signer.lambda.Invoke: calling handler <f

{"level": "debug", "filename": "connectionpool.py", "lineno": 396, "module": "connectionpool", "ts": "2019-09-30T14:29:35.851920Z", "msg": "https://lambda.us-east-1.amazonaws.com:443 \"POST /2015-03-31/functions/mind-01dadp74wfv607knpcb6vvxgtg/invocations HTTP/1.1\" 200 3042138"}
{"level": "debug", "filename": "parsers.py", "lineno": 234, "module": "parsers", "ts": "2019-09-30T14:29:35.853292Z", "msg": "Response headers: {'Date': 'Mon, 30 Sep 2019 14:30:17 GMT', 'Content-Type': 'application/json', 'Content-Length': '3042138', 'Connection': 'keep-alive', 'x-amzn-RequestId': '4eb66edf-ff05-40af-9613-4a7d8e93c0d7', 'x-amzn-Remapped-Content-Length': '0', 'X-Amz-Executed-Version': '$LATEST', 'X-Amzn-Trace-Id': 'root=1-5d92115d-b957f7527c2b1c3c87c74ea0;sampled=0'}"}
{"level": "debug", "filename": "parsers.py", "lineno": 234, "module": "parsers", "ts": "2019-09-30T14:29:35.855192Z", "msg": "Response headers: {'Date': 'Mon, 30 Sep 2019 14:30:17 GMT', 'Content-Type': 'application/json', 'Conten

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"3": {"segment0": [["Okay I think my join let's just started with that community on the language models right yep so."], "2019-09-30T10:29:13Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "ecc525a1a4d34f53aeaf2d97feada15e"], "segment1": [["Don't how any with having but engine General communities."], "2019-09-30T10:29:30Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "069ed6f0a2434222a5fc87c90c6dbe28"], "segment2": [["Yeah, it's was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally doesn't behavior. Yeah."], "2019-09-30T10:29:49Z", "ecfeeb757f0a4d47af1ebd513929264a", "74cf907c8c1d473abb406d002d35c3a9"], "segment3": [["Brand able to distinguis

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"3": {"segment0": [["Okay I think my join let's just started with that community on the language models right yep so."], "2019-09-30T10:29:13Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "ecc525a1a4d34f53aeaf2d97feada15e"], "segment1": [["Don't how any with having but engine General communities."], "2019-09-30T10:29:30Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "069ed6f0a2434222a5fc87c90c6dbe28"], "segment2": [["Yeah, it's was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally doesn't behavior. Yeah."], "2019-09-30T10:29:49Z", "ecfeeb757f0a4d47af1ebd513929264a", "74cf907c8c1d473abb406d002d35c3a9"], "segment3": [["Brand able to distinguis

In [90]:
## visualization
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
#m_time = formatTime("2019-09-16T09:53:21Z", True)
m_time = formatTime("2019-09-30T10:28:00Z", True)
for i in group_itr_2['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group_itr_2['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")





PIMs  3


Discussion:

 
Minutes from the start of the meeting:  0:21:13 ecc525a1a4d34f53aeaf2d97feada15e 

Okay I think my join let's just started with that community on the language models right yep so. 

Minutes from the start of the meeting:  0:21:30 069ed6f0a2434222a5fc87c90c6dbe28 

Don't how any with having but engine General communities. 

Minutes from the start of the meeting:  0:21:49 74cf907c8c1d473abb406d002d35c3a9 

Yeah, it's was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally doesn't behavior. Yeah. 

Minutes from the start of the meeting:  0:22:27 2c1b3cd9bfde44af956e7c0c7b3716fd 

Brand able to distinguish domain out of the and that case. 

Minutes from the start of the meeting:  0:22:33 78492bd0a4eb4de1810c98

In [75]:
## visualization
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
#m_time = formatTime("2019-09-16T09:53:21Z", True)
m_time = formatTime("2019-09-30T10:08:00Z", True)
for i in group['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")





PIMs  0


Discussion:

 
Minutes from the start of the meeting:  0:21:13 ecc525a1a4d34f53aeaf2d97feada15e 

Okay I think my join let's just started with that community on the language models right yep so. 

Minutes from the start of the meeting:  0:21:30 069ed6f0a2434222a5fc87c90c6dbe28 

Don't how any with having but engine General communities. 

Minutes from the start of the meeting:  0:21:49 74cf907c8c1d473abb406d002d35c3a9 

Yeah, it's was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally doesn't behavior. Yeah. 

Minutes from the start of the meeting:  0:22:27 2c1b3cd9bfde44af956e7c0c7b3716fd 

Brand able to distinguish domain out of the and that case. 

Minutes from the start of the meeting:  0:22:33 78492bd0a4eb4de1810c98